We will use this dataset to make a super team. One loophole in the analysis is the 'no budget constraint'. But what the heck, clubs are already spending 100 millions on players. We will not decide the formation now, lets see what players we get and then we can use appropriate formation. So, this amazing dataset provides 50 attributes to play with.
Below are the assumptions:
1) We have unlimited money to buy them (May be in coming kernels we can put a budget constraint)  
  

Unlike **[Part I][1]** we will not use traditional moneyball but use clustering using a useful python package- SciKit Learn.  
  
Age and work rate will be taken into the consideration in this model (this is an upgrade as in [Part I][1] we did measure their impact)


  [1]: https://www.kaggle.com/indrajit/making-a-super-team-i

Preamble  
========

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('../input/FullData.csv')
Names = pd.read_csv('../input/PlayerNames.csv')
df.assign(Index=np.nan)
df['Index'] = [v.split('/')[2] for v in Names['url']]
del df['Nationality']
del df['National_Position']
del df['National_Kit']
df.head()

,Name,Club,Club_Position,Club_Kit,Club_Joining,Contract_Expiry,Rating,Height,Weight,Preffered_Foot,...,Curve,Freekick_Accuracy,Penalties,Volleys,GK_Positioning,GK_Diving,GK_Kicking,GK_Handling,GK_Reflexes,Index
0,Cristiano Ronaldo,Real Madrid,LW,7.0,07/01/2009,2021.0,94,185 cm,80 kg,Right,...,81,76,85,88,14,7,15,11,11,20801
1,Lionel Messi,FC Barcelona,RW,10.0,07/01/2004,2018.0,93,170 cm,72 kg,Left,...,89,90,74,85,14,6,15,11,8,158023
2,Neymar,FC Barcelona,LW,11.0,07/01/2013,2021.0,92,174 cm,68 kg,Right,...,79,84,81,83,15,9,15,9,11,190871
3,Luis Suárez,FC Barcelona,ST,9.0,07/11/2014,2021.0,92,182 cm,85 kg,Right,...,86,84,85,88,33,27,31,25,37,176580
4,Manuel Neuer,FC Bayern,GK,1.0,07/01/2011,2021.0,92,193 cm,92 kg,Right,...,14,11,47,11,91,89,95,90,89,167495


Quantifying Age, Work Rate and BMI
==================================  
  
In this section we shall quantify below characteristics for every player:  
1) **Age** : Age from 24 to 28 shall be considered optimum. Player in this range will be given value '1' and remaining players will be given value '0'  
2) **Work Rate** : Average work rate for all the preferred positions will be taken.   
     a) High Work Rate: 10 Points  
     b) Low Work Rate: 0 Points  
     c) Medium Work Rate: 5 Points  
3) **BMI** : Body mass Index will be calculate with height and weight. It will have similar quantification to that of Work Rate.

Work Rate  
---------

TypeError: cannot do slice indexing on <class 'pandas.core.indexes.range.RangeIndex'> with these indexers [Name] of <class 'str'>

TypeError: cannot do slice indexing on <class 'pandas.core.indexes.range.RangeIndex'> with these indexers [Name] of <class 'str'>

In [19]:
work_rate_points = pd.DataFrame({'Work_Rate': df.Work_Rate.unique()})
work_rate_points['Work_Rate_Points'] = ['5','5','7.5','2.5','10','7.5','5','2.5','0']
df.merge(work_rate_points, on='Work_Rate', how='left')
df.head()

TypeError: cannot do slice indexing on <class 'pandas.core.indexes.range.RangeIndex'> with these indexers [Name] of <class 'str'>

,Name,Club,Club_Position,Club_Kit,Club_Joining,Contract_Expiry,Rating,Height,Weight,Preffered_Foot,...,Curve,Freekick_Accuracy,Penalties,Volleys,GK_Positioning,GK_Diving,GK_Kicking,GK_Handling,GK_Reflexes,Index
0,Cristiano Ronaldo,Real Madrid,LW,7.0,07/01/2009,2021.0,94,185 cm,80 kg,Right,...,81,76,85,88,14,7,15,11,11,20801
1,Lionel Messi,FC Barcelona,RW,10.0,07/01/2004,2018.0,93,170 cm,72 kg,Left,...,89,90,74,85,14,6,15,11,8,158023
2,Neymar,FC Barcelona,LW,11.0,07/01/2013,2021.0,92,174 cm,68 kg,Right,...,79,84,81,83,15,9,15,9,11,190871
3,Luis Suárez,FC Barcelona,ST,9.0,07/11/2014,2021.0,92,182 cm,85 kg,Right,...,86,84,85,88,33,27,31,25,37,176580
4,Manuel Neuer,FC Bayern,GK,1.0,07/01/2011,2021.0,92,193 cm,92 kg,Right,...,14,11,47,11,91,89,95,90,89,167495


,Name,Club,Club_Position,Club_Kit,Club_Joining,Contract_Expiry,Rating,Height,Weight,Preffered_Foot,...,Curve,Freekick_Accuracy,Penalties,Volleys,GK_Positioning,GK_Diving,GK_Kicking,GK_Handling,GK_Reflexes,Index
0,Cristiano Ronaldo,Real Madrid,LW,7.0,07/01/2009,2021.0,94,185 cm,80 kg,Right,...,81,76,85,88,14,7,15,11,11,20801
1,Lionel Messi,FC Barcelona,RW,10.0,07/01/2004,2018.0,93,170 cm,72 kg,Left,...,89,90,74,85,14,6,15,11,8,158023
2,Neymar,FC Barcelona,LW,11.0,07/01/2013,2021.0,92,174 cm,68 kg,Right,...,79,84,81,83,15,9,15,9,11,190871
3,Luis Suárez,FC Barcelona,ST,9.0,07/11/2014,2021.0,92,182 cm,85 kg,Right,...,86,84,85,88,33,27,31,25,37,176580
4,Manuel Neuer,FC Bayern,GK,1.0,07/01/2011,2021.0,92,193 cm,92 kg,Right,...,14,11,47,11,91,89,95,90,89,167495
